<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium

In [ ]:
# Assuming 'trips.xlsx' is uploaded to your Colab environment
df_trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header
df_trips.sample(2)

In [ ]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips['Сoordinates'] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.')
df_trips[['Longitude', 'Latitude']] = df_trips['Сoordinates'].str.split('. ', expand=True)
df_trips.drop(['Сoordinates'], axis=1, inplace=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], errors='coerce')

df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

df_trips

In [ ]:
# Преобразование поездок в геометрию
# df_trips['geometry'] = df_trips.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
# df_trips_gdf = gpd.GeoDataFrame(df_trips, geometry='geometry')

In [ ]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus_lanes.geojson")
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")

In [ ]:
# Создаем траектории машин
def create_trajectories(df):
    trajs = []
    for car_id, group in df.groupby('Car_ID'):
        points = [Point(lon, lat) for lon, lat in zip(group['Longitude'], group['Latitude'])]
        trajs.append({'Car_ID': car_id, "geometry": LineString(points)})
    return gpd.GeoDataFrame(trajs, crs="EPSG:4326")

gdf_trips = create_trajectories(df_trips)

In [ ]:
# Добавляем buffer(0.0003) к выделенным полосам - примерно 30 метров
gdf_bus_lanes["geometry"] = gdf_bus_lanes["geometry"].buffer(0.0003)

# Определяем пересечения с учетом buffer
gdf_trips["on_bus_lane"] = gdf_trips.intersects(gdf_bus_lanes.unary_union)

In [ ]:


# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby("№_машины")["Дата_Время"].shift(-1)
df_trips["next_lon"] = df_trips.groupby("№_машины")["Долгота"].shift(-1)
df_trips["next_lat"] = df_trips.groupby("№_машины")["Широта"].shift(-1)

# Вычисление времени и расстояний
from geopy.distance import geodesic

def compute_speed(row):
    if pd.isnull(row["next_time"]):
        return None
    distance = geodesic((row["Широта"], row["Долгота"]), (row["next_lat"], row["next_lon"])).meters
    time_diff = (row["next_time"] - row["Дата_Время"]).total_seconds()
    return distance / time_diff if time_diff > 0 else None

df_trips["speed"] = df_trips.apply(compute_speed, axis=1)

# Средняя скорость на выделенных полосах и вне их
avg_speed_bus_lane = 25  # Можно получить из данных общественного транспорта
avg_speed_normal = df_trips["speed"].median()

# Оценка выигрыша во времени
df_trips["predicted_time"] = df_trips.apply(
    lambda row: row["distance"] / avg_speed_bus_lane if row["on_bus_lane"] else row["distance"] / row["speed"],
    axis=1
)
df_trips["time_saved"] = df_trips["time"] - df_trips["predicted_time"]

# Итоговый выигрыш во времени
total_time_saved = df_trips["time_saved"].sum()

print(f"Общая экономия времени: {total_time_saved / 3600:.2f} часов")
